# 深度学习计算5_GPU计算


In [1]:
!nvidia-smi

Wed Feb 02 16:29:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 442.23       Driver Version: 442.23       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 2060   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   27C    P8     9W /  N/A |    164MiB /  6144MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## 计算设备

In [3]:
import torch
from torch import nn

torch.cuda.is_available() # 输出 True

D:\Anaconda\envs\torch\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\envs\torch\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
D:\Anaconda\envs\torch\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


True

In [4]:
torch.cuda.device_count() # 输出1

1

In [5]:
torch.cuda.current_device() # 输出0

0

In [6]:
torch.cuda.get_device_name(0) 

'GeForce RTX 2060'

## Tensor的GPU计算   

默认情况下，`Tensor`会被存在内存上。因此，每次打印`Tensor`的时候看不到`GPU`相关标识。

In [7]:
x = torch.tensor([1,2,3]) 
x.device

device(type='cpu')

使用`.cuda()`可以将`CPU`上的`Tensor`转换（复制）到`GPU`上。      

如果有多块`GPU`，我们用`.cuda(i)`来表示第 `i` 块`GPU`及相应的显存（`i`从0开始）且`cuda(0)`和`cuda()`等价。

In [8]:
x = x.cuda(0)
x

tensor([1, 2, 3], device='cuda:0')

In [10]:
x.device

device(type='cuda', index=0)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

x = torch.tensor([1,2,3],device = device)
# or 
#x = torch.tensor([1,2,3]).to(device)
x

tensor([1, 2, 3], device='cuda:0')

In [14]:
# 如果对在GPU上的数据进行运算，那么结果还是存放在GPU上。
y = x**2
y

tensor([1, 4, 9], device='cuda:0')

需要注意的是，存储在不同位置中的数据是不可以直接进行计算的。       

即**存放在CPU上的数据不可以直接与存放在GPU上的数据进行运算**，位于**不同GPU**上的数据也是不能直接进行计算的。

In [17]:
# z = y+x.cpu()
# 会出错：Expected all tensors to be on the same device

## 模型的GPU计算

同`Tensor`类似，PyTorch模型也可以通过`.cuda`转换到GPU上。  

可以通过检查模型的参数的`device`属性来查看存放模型的设备。

In [18]:
net = nn.Linear(3,1)
list(net.parameters())[0].device

device(type='cpu')

In [20]:
list(net.parameters())

[Parameter containing:
 tensor([[ 0.1467, -0.4701, -0.1255]], requires_grad=True),
 Parameter containing:
 tensor([0.5032], requires_grad=True)]

In [21]:
net.cuda()
list(net.parameters())[0].device

device(type='cuda', index=0)

In [22]:
x = torch.rand(2,3).cuda()
net(x)

tensor([[0.2069],
        [0.1031]], device='cuda:0', grad_fn=<AddmmBackward>)